In [29]:
from azureml.core import Workspace, Experiment, Environment, Dataset
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.estimator import Estimator
import json

In [30]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [31]:
# load experiment cfg
with open("experiment_cfg.json", "r") as cfg_file:
    cfg = json.load(cfg_file)

## Prepare Script

In [32]:
%%writefile ./src/prepare.py

from azureml.core import Run
from argparse import ArgumentParser
import numpy as np
import pandas as pd
import datetime
import os

run = Run.get_context()

parser = ArgumentParser()
parser.add_argument('--output', dest='prepared_data')
args = parser.parse_args()

# load datasets
df_symptoms = run.input_datasets['symptomcodes'].to_pandas_dataframe()
df = run.input_datasets['df_raw'].to_pandas_dataframe()

###########################################################

# get only data from last t years
t = 5
df = df[df['Job Card.Date Start Work']>(datetime.datetime.today() - datetime.timedelta(days=t*365))]

############################################################

# clean data
df = df.replace(['', '0', '-', '000','N/A'], np.nan)
df = df.dropna().reset_index(drop=True)

#############################################################################

# combine Component/Failure Code in train data
df = pd.concat([df, pd.DataFrame(df.apply(lambda x: (x['Job Card.ComponentCode'],x['Job Card.FailureCode']), axis=1), columns=['CompFail'])], axis=1)

# combine Component/Failure Code in symptom table
df_symptoms = df_symptoms[['ComponentCode', 'FailureCode', 'Symptom1', 'Symptom2', 'Symptom3', 'Symptom4']]
df_symptoms = pd.concat([df_symptoms, pd.DataFrame(df_symptoms.apply(lambda x: (x['ComponentCode'],x['FailureCode']),axis=1), columns=['CompFail'])],axis=1)

# merge train data on symptoms
df = pd.merge(df, df_symptoms, on='CompFail', how='left')
df = pd.concat([df, pd.DataFrame(df[['Symptom1', 'Symptom2', 'Symptom3', 'Symptom4']].apply(lambda x: tuple([ x[col] for col in ['Symptom1','Symptom2','Symptom3','Symptom4'] if str(x[col]) != 'None' ]), axis=1), columns=['Symptoms'])], axis=1)

# remove rows with no symptoms
df = df[df['Symptoms']!=()]

##############################################################################

# merge into one row per case
df = df.groupby('Job Card.JobCard Number').apply(lambda x: pd.Series({
    'ProductId': ' '.join(x['Installed Base.InstalledBase ProductID'].unique()),
    'ProductNr': ' '.join(x['Product.Product Number'].unique()),
    'Symptoms': ' '.join(map(str, list(set(x['Symptoms'].sum())))),
    'Start': x['Job Card.Date Start Work'].min(),
    'End': x['Job Card.Date End Work'].max()
  })).reset_index()

###############################################################################

# compute duration column
df = pd.concat([df, pd.DataFrame((df['End'] - df['Start']), columns=['duration'])],axis=1)
df['duration'] = df['duration'].apply(lambda x: x.seconds / 3600)

##############################################################################

# save train and test data
path = arg.prepared_data if args.prepared_data else './outputs'
os.makedirs(path, exist_ok=True)
df.to_csv(path + '/prepared_data.csv', sep=';', header=True, index=False)

run.complete()

Overwriting ./src/prepare.py


## Create Estimator

In [33]:
est = Estimator(entry_script='prepare.py', source_directory='src', 
              inputs=[   ws.datasets[cfg['symptomcodes_dataset']].as_named_input('symptomcodes'), 
                         ws.datasets[cfg['raw_data_dataset']].as_named_input('df_raw')       ],
              compute_target=ws.compute_targets['mlcompute'], environment_definition=ws.environments[cfg['env_name']])

## Run Experiment

In [34]:
exp = Experiment(ws, cfg['experiment_name'])
run = exp.submit(est)
run.wait_for_completion(show_output=True)

RunId: SKLearnPrediction_1592566375_0c5f8716
Web View: https://ml.azure.com/experiments/SKLearnPrediction/runs/SKLearnPrediction_1592566375_0c5f8716?wsid=/subscriptions/793146d9-d4dc-4a73-9728-76c4ffd0cc0d/resourcegroups/rg_dynamics_test/workspaces/resdynml1test

Streaming azureml-logs/55_azureml-execution-tvmps_873964ec2c5ddac76d8b6d73b0a66fa8ff78833147906ea4f5875879a568ccff_d.txt

2020-06-19T11:36:04Z Executing 'Copy ACR Details file' on 10.0.0.4
2020-06-19T11:36:04Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
2020-06-19T11:36:04Z Starting output-watcher...
2020-06-19T11:36:04Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_c14e68a5a54beac144cd751fe11b91c5
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
8269c605f3f1: Waiting
6504d449e70c: Pulling fs layer
4e38f320d0d4: Pulling fs layer
b0a763e

{'runId': 'SKLearnPrediction_1592566375_0c5f8716',
 'target': 'mlcompute',
 'status': 'Completed',
 'startTimeUtc': '2020-06-19T11:36:06.363214Z',
 'endTimeUtc': '2020-06-19T11:41:59.806513Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '029b6858-3135-42e4-b665-6cb7b2fbaa4b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '02e6cb83-4d0c-42b2-bbef-e103c74b3a3c'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'df_raw', 'mechanism': 'Direct'}}, {'dataset': {'id': '88af5740-1a1b-4e09-8129-d3c538680909'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'symptomcodes', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'prepare.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'mlcompute',
  'dataReferences': {},
  'data': {'df_

## Register Datasets

In [35]:
run.download_file('outputs/prepared_data.csv', output_file_path='artifacts/prepared_data.csv')
ds = ws.datastores[cfg['storage']]
data_ref = ds.upload_files(['artifacts/prepared_data.csv'], target_path='./'+cfg['experiment_name'], overwrite=True)
prepared_data_dataset = Dataset.Tabular.from_delimited_files(data_ref, separator=';', header=True, infer_column_types=True)
prepared_data_dataset.register(ws, cfg['prepared_data_dataset'], create_new_version=True)

Uploading an estimated of 1 files
Uploading artifacts/prepared_data.csv
Uploaded artifacts/prepared_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "source": [
    "('workspaceblobstore', './SKLearnPrediction')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "79c10931-897b-434d-b889-8469c14e6556",
    "name": "SKLearnPreparedData",
    "version": 4,
    "workspace": "Workspace.create(name='resdynml1test', subscription_id='793146d9-d4dc-4a73-9728-76c4ffd0cc0d', resource_group='rg_dynamics_test')"
  }
}